In [1]:
def lire_fichier(didier):
    f = open(didier, encoding="utf-8")
    chaine = f.read()
    f.close()
    return chaine

import glob
documents = {}
chemin_corpus = "corpus_multi/fr/*/*"

for chemin in glob.glob(chemin_corpus):
    documents[chemin] = lire_fichier(chemin)

In [2]:
liste_doc = list(documents.keys())
#print(documents[liste_doc[23]])

In [3]:
def decouper_mots(chaine):
    mots = chaine.split()
    return mots
def creer_index(documents):
    print("création de l'index")
    index = {}
    for chemin, contenu in documents.items():
        mots = decouper_mots(contenu)
        vocabulaire = set(mots)
        for mot in vocabulaire:
            if mot not in index:
                index[mot] = []
            index[mot].append(chemin)
    return index

index = creer_index(documents)
print("Nbr de mots indexés :", len(index))

création de l'index
Nbr de mots indexés : 33654


In [4]:
def creer_index_inverse(documents):
    print("création de l'index inverse")
    index_inverse = {}
    for chemin, contenu in documents.items():
        mots = decouper_mots(contenu)
        dic_frequence = {}
        for mot in mots:   
            if mot not in dic_frequence:
                dic_frequence[mot]=0
            dic_frequence[mot]+=1
            #compter les occurrences dans dic
        index_inverse[chemin] = dic_frequence
    return index_inverse
index_inverse = creer_index_inverse(documents)
print("Nbr de textes indexés :", len(index_inverse))

création de l'index inverse
Nbr de textes indexés : 272


In [5]:
def indexer_requete(requete):
    index_requete = {}
    mots = decouper_mots(requete)
    for mot in mots:   
        if mot not in index_requete:
            index_requete[mot]=0
        index_requete[mot]+=1
    return index_requete
requete = "moteur vais tester moteur"
index_requete = indexer_requete(requete)
print(index_requete)

{'moteur': 2, 'vais': 1, 'tester': 1}


In [6]:
for mot in index_requete:   
    if mot in index:
        print(mot)
        print(index[mot])

moteur
['corpus_multi/fr/test/2009-02-20_celex_IP-09-300.fr.html', 'corpus_multi/fr/test/2009-01-28_celex_IP-09-141.fr.html', 'corpus_multi/fr/appr/2009-12-16_celex_IP-09-1950.fr.html', 'corpus_multi/fr/appr/2009-04-20_celex_IP-09-594.fr.html', 'corpus_multi/fr/appr/2009-01-19_celex_IP-09-67.fr.html', 'corpus_multi/fr/appr/2009-02-19_celex_IP-09-297.fr.html', 'corpus_multi/fr/appr/2009-06-19_celex_IP-09-967.fr.html', 'corpus_multi/fr/appr/2009-09-22_celex_IP-09-1341.fr.html', 'corpus_multi/fr/appr/2009-03-04_celex_IP-09-351.fr.html', 'corpus_multi/fr/appr/2009-05-06_celex_IP-09-704.fr.html', 'corpus_multi/fr/appr/2009-08-04_celex_IP-09-1221.fr.html', 'corpus_multi/fr/appr/2009-11-27_celex_IP-09-1837.fr.html', 'corpus_multi/fr/appr/2009-06-18_celex_IP-09-952.fr.html', 'corpus_multi/fr/appr/2009-11-12_celex_IP-09-1703.fr.html', 'corpus_multi/fr/appr/2009-12-18_celex_IP-09-1966.fr.html', 'corpus_multi/fr/appr/2009-01-28_celex_IP-09-131.fr.html', 'corpus_multi/fr/appr/2009-06-18_celex_IP-0

In [7]:
def requeter_documents(requete, index):
    index_requete = indexer_requete(requete)
    documents_trouves = []
    for mot in index_requete:
        if mot in index:
            documents_pertinents = index[mot]
            documents_trouves+=documents_pertinents
    return set(documents_trouves)
requete = "sensibilisation minorités"
documents_trouves =  requeter_documents(requete, index)

print(requete)
print("Nbr de documents trouvés :", len(documents_trouves))
requete = "liberté humaine"
documents_trouves =  requeter_documents(requete, index)
print(requete)
print("Nbr de documents trouvés :", len(documents_trouves))


sensibilisation minorités
Nbr de documents trouvés : 9
liberté humaine
Nbr de documents trouvés : 23


In [8]:
def calculer_similarite_cosinus(requete, index_inverse,documents_trouves):
    from scipy import spatial
    
    index_requete = indexer_requete(requete)#pour avoir la même structure de dico mot:effectif
    resultat = []# où on va stocker les docs et les similarités
    
    for doc in documents_trouves:
        #On fait l'union des mots du document en cours et de la requête :
        vocabulaire = set(index_requete.keys()).union(index_inverse[doc].keys())
        
        #On aura un vecteur pour chacun
        vecteur, vecteur_requete = [], []
        
        for mot in vocabulaire:
            if mot in index_inverse[doc]:#si le mot est dans le document
                vecteur.append(index_inverse[doc][mot])#on ajoute son effectif dans le vecteur
            else:
                vecteur.append(0)# Sile mot est absent de ce document, on ajoute un zéro
                
            if mot in index_requete:#Idem pour la requête
                vecteur_requete.append(index_requete[mot])
            else:
                vecteur_requete.append(0)#= le mot est absent de la requête
        dist=spatial.distance.cosine(vecteur,vecteur_requete )
        similarite = 1-dist
        resultat.append([similarite, doc])

    return resultat # en sortie, des paires [similarité, document]


In [9]:
requete = "moteur vais tester moteur"
index_requete = indexer_requete(requete)
documents_trouves =  requeter_documents(requete, index)
resultat = calculer_similarite_cosinus(requete, index_inverse, documents_trouves)
for res in sorted(resultat, reverse=True):
    print(res)

requete = "la solidarité"
index_requete = indexer_requete(requete)
documents_trouves =  requeter_documents(requete, index)
resultat = calculer_similarite_cosinus(requete, index_inverse, documents_trouves)
for res in sorted(resultat, reverse=True):
    print(res)

ModuleNotFoundError: No module named 'scipy'

In [ ]:
#On voit dans la cellule ci-dessus que l'on a copié-collé pas mal de choses
#Maintenant nous pouvons créer une fonction qui fait toutes ces opérations 

def traiter_requete(requete, index, index_inverse):
    index_requete = indexer_requete(requete)
    documents_trouves =  requeter_documents(requete, index)
    resultat = calculer_similarite_cosinus(requete, index_inverse, documents_trouves)
    return resultat

    
for requete in ["la solidarité", "toto titi", "droit travail"]:
    resultat = traiter_requete(requete, index, index_inverse)
    print("Pour la requête %s (%i résultats)"%(requete, len(resultat)))
    for res in sorted(resultat, reverse=True)[:10]:#On se limite aux dix premiers résultats
        print(res)

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
### On voit que la présence de "la" dans une des requêtes donne des résultats étrange
### La suite ce sera de pondérer les termes avec  le tf-idf 

In [ ]:
def ponderation_tfidf(index):
    import math
    #On a besoin de connaître le nombre de documents dans le corpus
    all_documents = []
    for liste_doc in index.values():
        all_documents+=liste_doc
    Nb_documents = len(set(all_documents))
    
    #on reconstruit un index pondéré
    index_tf_idf = {}
    for mot, liste in index.items():
        index_tf_idf[mot] = math.log(Nb_documents/len(liste))
        #index_tf_idf[mot]=0
    return index_tf_idf

def calculer_similarite_cosinus_tf_idf(requete, index_inverse, documents_trouves, index):
    from scipy import spatial
    
    index_requete = indexer_requete(requete)#pour avoir la même structure de dico mot:effectif
    resultat = []# où on va stocker les docs et les similarités
    index_tf_idf = ponderation_tfidf(index)
    for doc in documents_trouves:
        #On fait l'union des mots du document en cours et de la requête :
        vocabulaire = set(index_requete.keys()).union(index_inverse[doc].keys())
        
        #On aura un vecteur pour chacun
        vecteur, vecteur_requete = [], []
        
        for mot in vocabulaire:
            if mot in index_inverse[doc]:#si le mot est dans le document
                #on ajoute son effectif (tf) pondéré par l'idf :
                vecteur.append(index_inverse[doc][mot]*index_tf_idf[mot])
            else:
                vecteur.append(0)# Sile mot est absent de ce document, on ajoute un zéro
                
            if mot in index_requete:#Idem pour la requête
                vecteur_requete.append(index_requete[mot])
            else:
                vecteur_requete.append(0)#= le mot est absent de la requête
        dist=spatial.distance.cosine(vecteur,vecteur_requete )
        similarite = 1-dist
        resultat.append([similarite, doc])

    return resultat # en sortie, des paires [similarité, document]

In [ ]:
def traiter_requete_tf_idf(requete, index, index_inverse):
    index_requete = indexer_requete(requete)
    documents_trouves =  requeter_documents(requete, index)
    resultat = calculer_similarite_cosinus_tf_idf(requete, index_inverse, documents_trouves, index)
    return resultat

    
for requete in ["la solidarité", "toto titi", "droit travail"]:
    resultat = traiter_requete_tf_idf(requete, index, index_inverse)
    print("Pour la requête %s (%i résultats)"%(requete, len(resultat)))
    for res in sorted(resultat, reverse=True)[:10]:#On se limite aux dix premiers résultats
        print(res)

NameError: name 'calculer_similarite_cosinus_tf_idf' is not defined